<h1 align="center">Model Versioning Example Notebook</h1>

In this guide, we will demonstrate how to use model versioning in the Arthur platform. We'll use the credit dataset (and a pre-trained model) to onboard 3 new models to the Arthur platform and put them together in the same Model Group.

In [ ]:
from arthurai import ArthurAI
from arthurai.common.constants import InputType, OutputType, Stage
import joblib
import numpy as np
import pandas as pd
from datetime import datetime, timedelta
import pytz

In [ ]:
import sys
sys.path.append("..")
from model_utils import load_datasets

#### Set up connection
Supply your API Key below to authenticate with the platform.

In [ ]:
# connect to Arthur
# UNCOMMENT the two lines below and enter your details
arthur = ArthurAI(
    # url="https://app.arthur.ai",  # you can also pass this through the ARTHUR_ENDPOINT_URL environment variable
    # login="<YOUR_USERNAME_OR_EMAIL>",  # you can also pass this through the ARTHUR_LOGIN environment variable
)

# Model v1 (Logistic Regression)

## Create Model

Creating the first model in a group requires no extra effort. A model group is created automatically when you create the model. Let's start with a logistic regression model trained on credit card data.

### Loading the Data

In [ ]:
(X_train, Y_train), (X_test, Y_test) = load_datasets("../fixtures/datasets/credit_card_default.csv")

# load our pre-trained classifier so we can generate predictions
sk_model = joblib.load("../fixtures/serialized_models/credit_lr.pkl")  # Logistic Regression pickle file

# get model predictions
preds = sk_model.predict_proba(X_train)
X_train["prediction_1"] = preds[:, 1]
X_train["prediction_0"] = preds[:, 0]

# get ground truth labels
X_train["gt_1"] = Y_train
X_train["gt_0"] = 1-Y_train

### Registering the Model

We'll instantiate a model object with a small amount of metadata about the model input and output types. Then, we'll extract the schema from the training data to build the complete model object.

In [ ]:
arthur_model_log_reg = arthur.model(partner_model_id=f"CreditRiskModel_QS_logistic_regression_{datetime.now().strftime('%Y%m%d%H%M%S')}_v1",
                                        display_name="Credit Risk",
                                        input_type=InputType.Tabular,
                                        output_type=OutputType.Multiclass)

prediction_to_ground_truth_map = {
    "prediction_0": "gt_0",
    "prediction_1": "gt_1"
}

arthur_model_log_reg.build(X_train,
                           pred_to_ground_truth_map=prediction_to_ground_truth_map,
                           positive_predicted_attr="prediction_1",
                           non_input_columns=['SEX'])

Since we know we'll want to make multiple versions of this model, let's give this version a label.

In [ ]:
arthur_model_log_reg.version_label = "logistic_regression"

Although a label is not neccessary, it can be helpful when you need to distinguish between several versions. Regardless of whether you give a version label or not, you will always be able to distinguish versions by an automatically assigned, incrementing `version_sequence_num`. The sequence number starts at 1 and each version in the group is assigned the next natural number, e.g. 1, 2, 3, and so on.

Although the model has been all but saved to Arthur, you'll notice that there is still no Model Group ID associated with this model...

In [ ]:
arthur_model_log_reg.model_group_id

Since we don't have a model group we want to assign this model to we can leave it as `None`. A model group will automatically be created with it when you call save.

In [ ]:
log_reg_model_id = arthur_model_log_reg.save()
log_reg_model_id

Now that the model has been saved, we have access to the information about model_group associated with it.  We retrieve it by passing in the model into the `arthur.get_model_group()` function.  Alternatively, we could pass in the model_group_id directly with `arthur.get_model_group(model_group_id)`.

In [ ]:
model_group = arthur.get_model_group(arthur_model_log_reg)
model_group

## Sending Inferences

Let's make some predictions with our model and send them to Arthur.

In [ ]:
from arthurai.core.decorators import log_prediction

@log_prediction(arthur_model_log_reg)
def model_predict(input_vec):
 return sk_model.predict_proba(input_vec)[0]

# 10 timestamps over the last week
timestamps = pd.date_range(start=datetime.now(pytz.utc) - timedelta(days=7),
                           end=datetime.now(pytz.utc),
                           periods=10)

inference_ids = {}
for timestamp in timestamps:
    for i in range(np.random.randint(50, 100)):
        datarecord = X_test.sample(1)  # fetch a random row
        prediction, inference_id = model_predict(datarecord, inference_timestamp=timestamp)  # predict and log
        inference_ids[inference_id] = datarecord.index[0]  # record the inference ID with the Pandas index
    print(f"Logged {i+1} inferences with Arthur from {timestamp.strftime('%m/%d')}")

gt_df = pd.DataFrame({'partner_inference_id': inference_ids.keys(),
                      'gt_1': Y_test[inference_ids.values()],
                      'gt_0': 1 - Y_test[inference_ids.values()]})
_ = arthur_model_log_reg.update_inference_ground_truths(gt_df)

# Model v2 (Random Forest)

## Create Model

Let's say you decide you don't like the logistic regression model and you decide you want to use your new random forest model. We want to link this new implementation as a new version of the first model (i.e. put it inside the same model group), so let's do that.

To start, create the new model, just like before.  Remember, you will need to send new inferences, reference data, etc.

In [ ]:
(X_train, Y_train), (X_test, Y_test) = load_datasets("../fixtures/datasets/credit_card_default.csv")

# load our pre-trained classifier so we can generate predictions
sk_model = joblib.load("../fixtures/serialized_models/credit_rf.pkl")  # Random Forest pickle file

# get model predictions
preds = sk_model.predict_proba(X_train)
X_train["prediction_1"] = preds[:, 1]
X_train["prediction_0"] = preds[:, 0]

# get ground truth labels
X_train["gt_1"] = Y_train
X_train["gt_0"] = 1-Y_train

Once again, let's build the Arthur Model wrapper and give it some updated information.

In [ ]:
arthur_model_rand_forest = arthur.model(partner_model_id=f"CreditRiskModel_QS_random_forest_{datetime.now().strftime('%Y%m%d%H%M%S')}_v2",
                                            display_name="Credit Risk v2 (Random Forest)",
                                            input_type=InputType.Tabular,
                                            output_type=OutputType.Multiclass)

prediction_to_ground_truth_map = {
    "prediction_0": "gt_0",
    "prediction_1": "gt_1"
}

arthur_model_rand_forest.build(X_train,
                               pred_to_ground_truth_map=prediction_to_ground_truth_map,
                               positive_predicted_attr="prediction_1",
                               non_input_columns=['SEX'])

### Set the new model to be a new version of the first model

Since we want to associate this new model with our previous model, we want to simply set the model_group_id on the model _before_ we call `.save()`.

In [ ]:
arthur_model_rand_forest.model_group_id = model_group.id

Alternatively, you could call `add_version` on `model_group` like so...

In [ ]:
# model_group.add_version(arthur_model_rand_forest)

Once we save the model, the Arthur platform will add this new model to the same model group as the first model and assign it a sequence number of 2. Of course, to help us distinguish more clearly between versions 1 and 2, let's give this the label, "random_forest".

In [ ]:
arthur_model_rand_forest.version_label = "random_forest"

In [ ]:
rand_forest_model_id = arthur_model_rand_forest.save()
rand_forest_model_id

## Sending Inferences

Let's send some inferences with this new model.

In [ ]:
from arthurai.core.decorators import log_prediction

@log_prediction(arthur_model_rand_forest)
def model_predict(input_vec):
 return sk_model.predict_proba(input_vec)[0]

# 10 timestamps over the last week
timestamps = pd.date_range(start=datetime.now(pytz.utc) - timedelta(days=7),
                           end=datetime.now(pytz.utc),
                           periods=10)

inference_ids = {}
for timestamp in timestamps:
    for i in range(np.random.randint(50, 100)):
        datarecord = X_test.sample(1)  # fetch a random row
        prediction, inference_id = model_predict(datarecord, inference_timestamp=timestamp)  # predict and log
        inference_ids[inference_id] = datarecord.index[0]  # record the inference ID with the Pandas index
    print(f"Logged {i+1} inferences with Arthur from {timestamp.strftime('%m/%d')}")

gt_df = pd.DataFrame({'partner_inference_id': inference_ids.keys(),
                      'gt_1': Y_test[inference_ids.values()],
                      'gt_0': 1 - Y_test[inference_ids.values()]})
_ = arthur_model_rand_forest.update_inference_ground_truths(gt_df)

# Model v3 ("Fair" Random Forest)

## Create Model

Once again, you want to iterate on your model because you feel like the model is biased.  Let's onboard a model v3 that is more fair...or at least we hope it is!

In [ ]:
(X_train, Y_train), (X_test, Y_test) = load_datasets("../fixtures/datasets/credit_card_default.csv")

# load our pre-trained classifier so we can generate predictions
sk_model = joblib.load("../fixtures/serialized_models/credit_frf.pkl")  # "Fair" Random Forest pickle file

# get model predictions
preds = sk_model.predict_proba(X_train)
X_train["prediction_1"] = preds[:, 1]
X_train["prediction_0"] = preds[:, 0]

# get ground truth labels
X_train["gt_1"] = Y_train
X_train["gt_0"] = 1-Y_train

Same as before, set the model info and build it.

In [ ]:
arthur_model_fair_rf = arthur.model(partner_model_id=f"CreditRiskModel_QS_fair_random_forest_{datetime.now().strftime('%Y%m%d%H%M%S')}_v3",
                                        display_name="Credit Risk v3 (Fair Random Forest)",
                                        input_type=InputType.Tabular,
                                        output_type=OutputType.Multiclass)

prediction_to_ground_truth_map = {
    "prediction_0": "gt_0",
    "prediction_1": "gt_1"
}

arthur_model_fair_rf.build(X_train, pred_to_ground_truth_map=prediction_to_ground_truth_map, positive_predicted_attr="prediction_1", non_input_columns=['SEX'])

### Set the new model to be a part of the same model group

Again, let's set the model group. Last time we did it by setting the `model_group_id` directly. This time, let's use the `add_version` method just for funsies. We can also set the `version_label` here as well.

In [ ]:
model_group.add_version(arthur_model_fair_rf, label="fair_random_forest")

alternatively...

In [ ]:
# arthur_model_fair_rf.version_label = "fair_random_forest"
# arthur_model_fair_rf.model_group_id = model_group.id

and finally, let's save this one as well.

In [ ]:
fair_rf_model_id = arthur_model_fair_rf.save()
fair_rf_model_id

## Sending Inferences

And we should send some more inferences too right?

In [ ]:
from arthurai.core.decorators import log_prediction

@log_prediction(arthur_model_fair_rf)
def model_predict(input_vec):
 return sk_model.predict_proba(input_vec)[0]

# 10 timestamps over the last week
timestamps = pd.date_range(start=datetime.now(pytz.utc) - timedelta(days=7),
                           end=datetime.now(pytz.utc),
                           periods=10)

inference_ids = {}
for timestamp in timestamps:
    for i in range(np.random.randint(50, 100)):
        datarecord = X_test.sample(1)  # fetch a random row
        prediction, inference_id = model_predict(datarecord, inference_timestamp=timestamp)  # predict and log
        inference_ids[inference_id] = datarecord.index[0]  # record the inference ID with the Pandas index
    print(f"Logged {i+1} inferences with Arthur from {timestamp.strftime('%m/%d')}")

gt_df = pd.DataFrame({'partner_inference_id': inference_ids.keys(),
                      'gt_1': Y_test[inference_ids.values()],
                      'gt_0': 1 - Y_test[inference_ids.values()]})
_ = arthur_model_fair_rf.update_inference_ground_truths(gt_df)

# The Model Group

Now we can really get into some fun stuff!  I'll put the connection info here just in case you've decided to start in this section

In [ ]:
from arthurai import ArthurAI
from arthurai.common.constants import InputType, OutputType, Stage
import joblib
import numpy as np
import pandas as pd
from datetime import datetime, timedelta
import pytz

import sys
sys.path.append("..")
from model_utils import load_datasets

# connect to Arthur
# UNCOMMENT the two lines below and enter your details
arthur = ArthurAI(
    # url="https://app.arthur.ai",  # you can also pass this through the ARTHUR_ENDPOINT_URL environment variable
    # login="<YOUR_USERNAME_OR_EMAIL>",  # you can also pass this through the ARTHUR_LOGIN environment variable
)

# If you want to start by getting the model_group directly from an id, you can use this line of code below
# model_group = arthur.get_model_group('<model_group_uuid>')

Let's briefly explore some of the things we can do when playing around with model groups.  You can get properties such as the id...

In [ ]:
model_group.id

or the name.

In [ ]:
model_group.name

If you don't like that name, you can change it!

In [ ]:
model_group.name = 'Awesome Credit Risk Group'
model_group.name

Same groes for the description as well.

In [ ]:
model_group.description = 'Super awesome collection of credit risk models!'
model_group.description

Those changes will instantly be reflected on the Arthur Platform as well.

In [ ]:
arthur.get_model_group(model_group.id)

Now that we have the model group information, we can take a look at a list of versions in the group.

In [ ]:
# we can see the names of the 3 models we created above
versions = model_group.get_versions()
for version in versions:
    print(version.display_name)

If we want to retrieve a specific model_version, then we simply need to specify which version want.

We can specify by sequence_num...

In [ ]:
model_group.get_version(sequence_num=2).display_name

and we can specify by label.

In [ ]:
model_group.get_version(label="fair_random_forest").display_name

If you really want to get funky, you can start running comparisons between models.  You can get a visualizer by using `model_group.viz()`.  If you want to specify only a subset of versions in the group, you can specify a list of `sequence_nums` or `labels`. as arguments in the `viz()` method.

In [ ]:
viz = model_group.viz()

Now that we have our visualizer set up, lets take a look at a metric series.  Here we see two accuracy metrics shown over time. Throughout this notebook, we sent predictions over a weeklong time horizon. and we can see how these metrics did over time.  The red lines represent Area Under the Curve (AUC) for the models while the blue lines represents False Positive Rate. The lighter the color is, the more recent the model version.

Feel free to display other accuracy metrics on the graph by editing the line below. Up to 10 metrics can be supported at one time.

In [ ]:
viz.metric_series(["auc", "falsePositiveRate"], time_resolution="day")

We can also look at a drift series.  This graph shows how much drift we're seeing across the 3 models on the "PAY_0" attribute. The newest model is shown with the darkest line while the oldest has the lightest line. Since all of these models use the same train/test data, we expect to see these 3 lines overlap quite a bit.

Feel free to play around with the drift metric or the variable to show drift on. Like the metric series, up to 10 variables are supported to show data drift!

In [ ]:
viz.drift_series(["PAY_0"], drift_metric="KLDivergence", time_resolution="hour")

#### Housekeeping

This section exists solely to aide automated testing cleanup.  Feel free to ignore if you are a human.

In [ ]:
model_id = log_reg_model_id  # This is only needed to help with automated model cleanup